### Django authentication system
- 따로 settings에 추가 필요 없음
- 이미 들어가있음('django.contrib.auth',)

- 인증(Authentication)
  - 신원확인
  - 사용자가 자신이 누구인지 확인하는 것
- 권한(Authorization)
  - 권한 부여
  - 인증된 사용자가 수행할 수 잇는 작업을 결정

- 두 번째 앱으로 처리
- accounts -> 회원, 인증

- auth와 관련된 경로나 키워드들을 Django 내부적으로 accounts라는 이름으로 사용하고 있기 때문에 되도록 accounts로 지정하는 것을 권장
- 다른 이름으로 설정해도 되지만 나중에 추가 설정을 필요로 함.(번거로워짐)


### 커스텀 유저 모델로 대체하기
- Django는 기본 user model을 필수적으로 custom user model로 대체
- 기본 User모델로 충분하더라도 커스텀 하도록 "강력하게" 권장
- 대체 커스텀은 migration 하기 전에 마쳐야 함
- 일부 프로젝트에서는 user-model의 기본 인증 요구사항이 적절하지 않을 수 있음
- username 대신 email을 식별로 이용하는 경우 built-in user model에서는 수정하기 쉽지 않다. 따라서 처음부터 커스텀해서 시작하라는 것.

- AbstractUser를 상속받는 커스텀 user 클래스 작성
- 기존 user도 AbstractUser를 상속받기 때문에 이것만 상속받으면 커스텀할 수 있음



### AUTH_USER_MODEL
- 이 부분을 custom user 모델로 바꿔주어야 하는데 안보임
- settings는 global settings를 상속받는데 이 global settings에 있음
- 대체하는 과정을 외우기 어렵다면 공식문서 보고 그대로 따라가는게 좋음
- 1단계:
``` python
# accounts/models.py
from django.db import models
from django.contrib.auth.models import AbstractUser

class User(AbstractUser):
    pass
```

- 2단계:
``` python
# settings.py
AUTH_USER_MODEL = 'accounts.User'
```

- 3단계:
``` python
# accounts/admin.py
from django.contrib import admin
from django.contrib.auth.admin import UserAdmin
from .models import User

# Register your models here.
admin.site.register(User, UserAdmin)
```

### AbstractUser
- 관리자 권한과 함께 완전한 기능을 가지고 있는 User model을 구현하는 추상 기본클래스
- 추상 기본 클래스(Abstract base classses)란 클래스로만 존재함
- 몇가지 공통정보를 여러 다른 모델에 넣을 때 사용하는 클래스
- 데이터베이스 테이블을 만드는데 사용되지 않으며 다른 모델의 기본 클래스로 상요됨


### 주의사항
- 프로젝트 중간에 AUTH_USER_MODEL 변경시 훨~씬 어려운 작업이 필요

### DB 초기화
1. migrations 파일 삭제(폴더 안의 숫자가 붙은(0001, 0002 등) 파일만 삭제)
2. db.sqlite3 삭제
3. migrations 진행(makemigrations, migrate)

- 이때 만들어진 테이블 이름이 auth_user였는데 account_user로 바뀜

### HTTP Cookies
- HTTP 특징
  1. 비 연결 지향
    - 예를 들어 네이버는 우리에게 네이버 메인페이지를 응답하고 끝. 연결이 되어있는것은 아님
  2. 무상태
    - 연결을 끊는 순간 통신이 끝나며 상태 정보가 유지되지 않음
    - 클라이언트와 서버가 주고받은 메세지들은 독립적

- 그럼에도 불구하고 다른 페이지로 갔을때 로그인 상태가 유지됨
- 서버와 클라이언트간 지속적인 상태 유지를 위해 "쿠키와 세션"이 존재

### 쿠키
- HTTP 쿠키는 상태가 있는 세션을 만들도록 해 줌
- 쿠키 중에서 상태를 유지시켜 주는 것을 세션 또는 세션 쿠키 라고 함
- 서버가 사용자의 웹브라우저에 전송하는 작은 데이터 조각(응답에 쿠키가 있음)
- 사용자가 웹사이트를 방문할경우 웹사이트의 서버를 통해 사용자의 컴퓨터에 설치되는 작은 기록 정보 파일
  - 브라우저는 쿠키를 로컬에 KEY-VALUE형식으로 저장
  - 이렇게 쿠키를 저장해놓았다가 동일한 서버에 재요청시 저장된 쿠키를 함꼐 전송
- 쿠키는 두 요청이 동일한 브라우저에서 들어왔는지 아닌지 판단할때 주로 사용됨
  - 이를 이용해 사용자의 로그인 상태를 유지할 수 있음
  - 상태가 없는 HTTP 프로토콜에서 상태 정보를 기억시켜주기 때문
- 즉 웹페이지에 접속하면 웹 페이지를 응답한 서버로부터 쿠키를 받아 브라우저에 저장했다가 같은 서버에 재요청시마다 쿠키도 함께 보낸다.



### 쿠키 사용 목적
1. 세션 관리
   - 로그인, 아이디 자동완성, 공지 하루 안 보기, 팝업체크, 장바구니 등의 정보관리
2. 개인화
   - 사용자 선호, 테마 등의 설정
3. 트래킹
   - 사용자 행동을 기록 및 분석

### 세션
- 사이트와 특정 브라우저 사이의 상태를 유지시키는 것
- 클라이언트가 서버에 접속하면 서버가 특정 session id를 발급하고, 클라이언트는 session id를 쿠키에 저장
  - 클라이언트가 다시 동일한 서버에 접속하면 요청과 함계 쿠키(session id가 저장된)를 서버에 전달
  - 쿠키는 요청때마다 서버에 함께 전송되므로 서버에서 session id를 확인해 알맞은 로직을 처리
- session id는 세션을 구별하기 위해 필요하며, 쿠키에는 session id만 저장(핵심 정보는 서버가 들고있음)

### 쿠키 Lifetime(수명)
1. Session cookie
   - 현재 세션이 종료되면서 삭제됨 (세션이 만료되었습니다.)
   - 브라우저 종료와 함께 세션이 삭제됨
2. persistent cookies
   - Expires 속성에 지정된 날짜 혹은 Max-Age 속성에 지정된 기간이 지나면 삭제됨

### Session in Django
- Django는 database-backed sessions 저장 방식을 기본값으로 사용
  - session 정보는 Django DB의 django_session 테이블에 저장
  - 설정을 통해 다른 저장방식으로 변경 가능
- Django는 특정 session id를 포함하는 쿠키를 사용해서 각각의 브라우저와 사이트가 연결된 session을 알아냄
- Django는 우리가 session 메커니즘(복잡한 동작원리)에 대부분을 생각하지 않아도 되게끔 많은 도움을 줌


### Authentication in Web requests
- Django가 제공하는 인증 관련 built-in forms(인증은 많은 개발이 필요하기에 기본 폼을 활용, 대부분 Django 지원)

### Login
- 로그인은 Session을 Create 하는 과정
- AuthenticationForm
  - 로그인을 위한 빌트인 폼
  - 로그인하고자 하는 사용자 정보를 입력받고 기본적으로 username과 password를 받아 데이터가 유효한지 검증
  - request를 첫번째 인자로 취함(명심할것)
- login 함수
  - login(request, user, backend=None)
  - 인증된 사용자를 로그인 시키는 로직으로 view 함수에서 사용됨
  - 현재 세션에 연결하려는 인증 된 사용자가 있을 경우 사용
  - HttpRequest 객체와 User 객체가 필요
- get_user() 
  - AuthenticationForm의 인스턴스 메서드
  - 유효성 검사를 통과했을 경우 로그인한 사용자 객체를 반환

### Authentication with user
- 현재 로그인 되어있는 유저 정보 출력하기
- context에 user가 없음에도 {{ user }}가 먹힘
- settings.py의 context_processors 설정 때문
  - 기본적으로 템플릿에서 사용 가능한 변수들이 포함됨
  - 현재 user 변수를 담당하는 프로세서는 django.contrib.auth.context_processors.auth
  - 로그인하지 않은 유저는 AnonymousUser, 로그인한 유저는 User

### Logout
- 로그아웃은 Session을 Delete하는 과정
- 클라이언트와 서버에 있는 것 모두 삭제(한쪽만 지워져도 로그아웃 되지만 둘 다 지움)
- HttpRequest 객체를 인자로 받고 반환값이 없음
- 사용자가 로그인하지 않아도 오류를 발생시키지는 않음
- 다음 2가지 일을 처리한다.
  1. 현재 요청에 대한 session data를 DB에서 삭제
  2. 클라이언트의 쿠키에서도 sessionid를 삭제
   - 이는 다른 사람이 동일한 웹 브라우저를 사용하여 로그인하고, 이전 사용자의 세션 데이터에 엑세슷 하는 것을 방지하기 위함

### 회원가입
- UserCreationForm
  - 권한이 없는 새 user를 생성하는 ModelForm
  - 3개의 필드를 가짐
    - username
    - password1
    - password2
- UserCreationForm 은 모델폼인데 user 모델을 기반으로 만들어졌음. 하지만 우리는 auth.user가 아닌 accounts.User로 바꿨으므로 그대로 실행하면 안됨.
  - 오버라이드 : UserCreationiForm 그대로 상속받아서 model만 우리가 원하는 모델로 바꿔주면 됨
  - 이때 User를 직접 참조하는것을 권장하지 않음(from .models import User 지양)
``` python
from django.contrib.auth import get_user_model
# 이걸로 할 것
# 사실 똑같은데 Django가 원하지 않음
```
- get_user_model()은 현재 프로젝트에서 활성화된 사용자 모델을 반환

### 회원 탈퇴
- DB에서 유저를 DELETE 하는 것이다.
- 세션까지 지우고 싶다면 탈퇴 후 로그아웃(로그아웃부터 하면 뭘 탈퇴하고싶은지 알 수 없음)

### 회원정보 수정
- UserChangeForm
  - 사용장의 정보 및 권한을 변경하기 위해 admin 인터페이스에서 사용되는 ModelForm
  - 이 또한 ModelForm이기때문에 instance 인자로 기존 user 데이터 정보를 받는 구조 또한 동일함
- 비밀번호 수정은 암호화가 필요하여 따로 다른 페이지로 이동하여 바꾸어야 한다.
- 바꿀 수 있는 정보가 너무 많으면 안됨(본인이 admin이 될 수 있음)
- 따라서 바꿀 필드를 몇개만 지정

### 비밀번호 수정
- PasswordChangeForm
  - 이전 비밀번호를 입력하여 비밀번호를 변경할 수 있도록 함
  - 이전 비밀번호를 입력하지 않고 비밀번호를 설정할 수 있는 SetPasswordForm을 상속받는 서브 클래스
- PasswordChangeForm는 필수 인자가 필요함(첫번째 인자 user)
- 비밀번호 변경시 기존 세션과 회원 인증 정보가 일치하지 않게 되어 로그인이 풀림
- 따라서 기존 세션을 새로운 세션으로 바꿔주는 과정이 필요함.
- 이것을 해주는게 update_session_auth_hash(request, user)
  - 현재 요청과 새 session data가 파생 될 업데이트 된 사용자 객체를 가져오고, 업데이트 해줌

### 로그인 사용자에 대한 접근 제한
- 로그인 상요자에 대해 접근을 제한하는 2가지 방법
  1. The raw way
     - is_authenticated attribute
  2. The login_required decorator(데코레이터인 것 주의!)

### is_authenticated
- User model의 속성 중 하나
- 모든 User 인스턴스에 대해 항상 True인 읽기 전용 속성
  - AnonymousUser에 대해서는 항상 False
- 일반적으로 request.user에서 이 속성을 사용
- 다만, 권한과는 관련이 없고 활성화 상태나 요효한 세션을 가지는지 등을 확인하지 않음. 그저 로그인했냐 안했냐만 판단
- html에서 보이는 것 만 가린다면 여전히 URL을 통해 이곳저곳 다 이동 가능
- views에서 아예 접근 못하게 막아야 함.

### 로그인된 사용자만 글 작성/수정/삭제
- 데코레이터 붙이면 로그인 안했을때 로그인창으로 돌려버림
  - 우리 주소를 어떻게 알고 로그인주소로 보내는가?
    - 주소는 모르지만 accounts/login/으로 보내도록 되어있음
    - 마침 우리가 이 주소를 쓰고있는 것.
  - 주소가 이상한데 어떤 뜻인가?
    - 원래 가려고 했던 주소를 남겨줌
    - 로그인 후 그 곳으로 바로 갈 수 있도록 하는것
    - 하지만 실제로는 안감 왜냐? login 하면 그냥 설정된대로 하기때문
    - 따라서, 하고싶다면 next가 붙은 그 주소로 보내야함
    - 주소를 작성하지 않으면 그 next가 붙은 주소로 보냄
    - next는 키값이고 value가 주소이므로 따로 받아줘서 처리해야 함.
- 두 데코레이터로 인해 발생되는 구조적 문제
  - delete에는 login_required를 붙이면 안됨(GET, POST 방식으로 인한 로직 막힘)
  - 따라서 함수 안에 로그인 확인하는 로직을 넣어야함